<a href="https://colab.research.google.com/github/impratiksingh/ElectionData/blob/master/Indian_General_Election_2019_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# These excel sheets are downloaded from the wesite of Election Commision of India . As the name suggest they contain historical data of previous
# General Elections

import pandas as pd
import numpy as np

ge_1998=pd.read_excel('GE_1998.xls')
ge_1999=pd.read_excel('GE_1999.xls')
ge_2004=pd.read_excel('GE_2004.xls')
ge_2009=pd.read_excel('GE_2009.xls')
ge_2014=pd.read_excel('GE_2014.xls')

In [385]:
ge_1998.head(2)

,ST_CODE,ST_NAME,PC_NO,PC_NAME,PC_TYPE,MONTH,YEAR,CAND_NAME,CAND_SEX,PARTYABBRE,TOTVOTPOLL,POSITION
0,S01,ANDHRA PRADESH,1,Srikakulam,GEN,3,1998,KINJARAPU YERRANNAIDU,M,TDP,286582,1
1,S01,ANDHRA PRADESH,1,Srikakulam,GEN,3,1998,APPAYYA DORA HANUMANTHU,M,NTRTDP(LP),200217,2


In [386]:
ge_1999.head(2)

,ST_CODE,ST_NAME,PC_NO,PC_NAME,PC_TYPE,MONTH,YEAR,CAND_NAME,CAND_SEX,PARTYABBRE,TOTVOTPOLL,POSITION
0,S01,ANDHRA PRADESH,1,Srikakulam,GEN,7,1999,Yerrannaidu Kinjarapu,M,TDP,373851,1
1,S01,ANDHRA PRADESH,1,Srikakulam,GEN,7,1999,Kanithi Viswanadaham,M,INC,276969,2


In [387]:
ge_2004.head(2)

,ST_CODE,ST_NAME,PC_NO,PC_NAME,PC_TYPE,MONTH,YEAR,CAND_NAME,CAND_SEX,CAND_CATEGORY,PARTYABBRE,TOTVOTPOLL,POSITION
0,S01,ANDHRA PRADESH,1,Srikakulam,GEN,3,2004,YERRANNAIDU KINJARAPU,M,GEN,TDP,361906,1
1,S01,ANDHRA PRADESH,1,Srikakulam,GEN,3,2004,KILLI KRUPARANI,F,GEN,INC,330027,2


In [388]:
ge_2009.head(2)

,ST_CODE,State name,Month,Year,PC Number,PC name,PC Type,Candidate Name,Candidate Sex,Candidate Category,Candidate Age,Party Abbreviation,Total Votes Polled,Position
0,S01,Andhra Pradesh,3,2009,1,Adilabad,ST,RATHOD RAMESH,M,ST,43,TDP,372268,1
1,S01,Andhra Pradesh,3,2009,1,Adilabad,ST,KOTNAK RAMESH,M,ST,39,INC,257181,2


In [389]:
ge_2014.head(2)

,ST_CODE,State name,Month,Year,PC Number,PC name,PC Type,Candidate Name,Candidate Sex,Candidate Category,Candidate Age,Party Abbreviation,Total Votes Polled,Position
0,S01,Andhra Pradesh,5,2014,1,Adilabad,ST,GODAM NAGESH,M,ST,49.0,TRS,430847,1
1,S01,Andhra Pradesh,5,2014,1,Adilabad,ST,NARESH,M,ST,37.0,INC,259557,2


In [0]:
# Though Candidate Category used to be a driving factor in past , but in recent elections we see the party name is more dominant factor , hence I am dropping out that columns
ge_2004=ge_2004.drop(['CAND_CATEGORY'],axis=1)

# 1998 , 1999 has similar structure so they can be combined as is
df_9899=pd.concat([ge_1998,ge_1999,ge_2004],ignore_index=True)

# 2009 , 2014 again has a similar structure and hence can be combined as is
df_0914=pd.concat([ge_2009,ge_2014],ignore_index=True)

# Dropping Candidate category 
df_0914=df_0914.drop(['Candidate Category'],axis=1)



In [391]:
df_9899.head(2)

,ST_CODE,ST_NAME,PC_NO,PC_NAME,PC_TYPE,MONTH,YEAR,CAND_NAME,CAND_SEX,PARTYABBRE,TOTVOTPOLL,POSITION
0,S01,ANDHRA PRADESH,1,Srikakulam,GEN,3,1998,KINJARAPU YERRANNAIDU,M,TDP,286582,1
1,S01,ANDHRA PRADESH,1,Srikakulam,GEN,3,1998,APPAYYA DORA HANUMANTHU,M,NTRTDP(LP),200217,2


In [0]:
# State Code , State Name , Contituency Number , Contituency Type , month of Election , Candidate's Name , Gender have not 
# been considered for analysis as they are very less determinant of a Candidate's Win.Total Votes polled will have relative impact , hence dropped.

df_9899.drop(['ST_CODE','ST_NAME','PC_NO','PC_TYPE','MONTH','CAND_NAME','CAND_SEX','TOTVOTPOLL'],axis=1,inplace=True)


In [393]:
df_9899.head(2)

,PC_NAME,YEAR,PARTYABBRE,POSITION
0,Srikakulam,1998,TDP,1
1,Srikakulam,1998,NTRTDP(LP),2


In [394]:
df_0914.head(2)

,ST_CODE,State name,Month,Year,PC Number,PC name,PC Type,Candidate Name,Candidate Sex,Candidate Age,Party Abbreviation,Total Votes Polled,Position
0,S01,Andhra Pradesh,3,2009,1,Adilabad,ST,RATHOD RAMESH,M,43.0,TDP,372268,1
1,S01,Andhra Pradesh,3,2009,1,Adilabad,ST,KOTNAK RAMESH,M,39.0,INC,257181,2


In [0]:
df_0914.drop(['ST_CODE','State name','Month','PC Number','PC Type','Candidate Name','Candidate Sex','Candidate Age','Total Votes Polled'],axis=1,inplace=True)

In [396]:
df_0914.head(2)

,Year,PC name,Party Abbreviation,Position
0,2009,Adilabad,TDP,1
1,2009,Adilabad,INC,2


In [397]:
list(df_9899.columns)

['PC_NAME', 'YEAR', 'PARTYABBRE', 'POSITION']

In [0]:
df_9899=df_9899.loc[:,['YEAR','PC_NAME','PARTYABBRE', 'POSITION']]

In [0]:
# Considering only Historical Results for Consideration , which Candidate got 1st Position , their Associated party and the Contituency Name.
col=['YEAR','PC','PARTY', 'POSITION']
df_9899.columns=col
df_0914.columns=col
df1=df_9899
df2=df_0914

In [400]:
# Combining the different datasets into one
election=pd.concat([df1,df2],ignore_index=True,sort=True)
print(election.shape)

(31697, 4)


In [0]:
# Created an Empty Dataset , which will be updated iteratively for different years.
# Reason being , I was unable to pivot the dataset as it was giving the multiple index error. I thought of 
# hadling it year on year basis

appended_data=pd.DataFrame(columns=pd.MultiIndex.from_product([['PARTY'], ['9999']],names=[None, 'YEAR']) , index=['ZZZ']).fillna(np.nan)
appended_data.index.name='PC'

# Taking only Winners' Data , iteratively for each Year
years=list(election.YEAR.unique())
for year in years:
  yearly_election=election[(election.POSITION==1) & (election.YEAR==year)]
  yearly_election=yearly_election.apply(lambda x : x.astype(str).str.upper().str.strip()) # UpperCase is done for consistency , strip for handling : 'ADILABAD ' and 'ADILABAD' like entries
  
  # I found duplicate contituency names , and they hindered pivoting , so dropped them (they were few in numbers , so not much data loass in dropping Ex: Amravati in MP and Maharashtra both)
  yearly_election=yearly_election.drop(list(yearly_election[yearly_election.duplicated(['PC'])].index))
  
  yearly_election=yearly_election[['PARTY','PC','YEAR']]
  yearly_election=yearly_election.pivot(index='PC', columns='YEAR')
 
  
  appended_data=appended_data.merge(yearly_election,how='outer',left_index=True,right_index=True)

# These were added to create the similar structured dataset, coudnt think of a better way today
appended_data.drop(columns=['9999'],level=1,inplace=True)
appended_data.drop(index=['ZZZ'],axis=0,inplace=True)

In [402]:
appended_data.head()

PARTY                    
YEAR        1998 1999 2004 2009 2014
PC                                  
ADILABAD     TDP  TDP  TRS  TDP  TRS
ADOOR        CPI  INC  CPI  NaN  NaN
AGRA         BJP   SP   SP  BJP  BJP
AHMADNAGAR   NaN  NaN  NaN  BJP  BJP
AHMEDABAD    BJP  BJP  BJP  NaN  NaN

**APPROACH 1**

In [0]:
'''
In order to wrap it up quickly , since data issues took most of the time , I am doing it in a non recursive manner , will use more elegant methods in next update.
I assumed that the recent winning party has more chance of winning and a party which won from a certain area in say 1998 will have exponentially low impact today.
Since the voter age group and mindset has changed so I assume an exponential shift in it.

Using Exponential Weighted Effect on the party's performance with time.

There are many name changes , and sub divisions done in the parliamentary contituencies. 
Ex : 'BARODA' : Vadodara; Ahmednagar to Ahmadnagar; Ahmedabad split into ahmedabad east and west. 
'ALLEPPEY' became 'ALAPPUZHA'. Trivandrum etc.In tomorrow's results even our own Gurgaon will be called Gurugram and Allahabad as Prayagraj.

To handle them , I took the Exponential weighing .

'''
from collections import Counter
d_1998=dict(Counter(appended_data.PARTY['1998']))
d_1999=dict(Counter(appended_data.PARTY['1999']))
d_2004=dict(Counter(appended_data.PARTY['2004']))
d_2009=dict(Counter(appended_data.PARTY['2009']))
d_2014=dict(Counter(appended_data.PARTY['2014']))

d_1998={k:v*2**1 for k,v in d_1998.items()}
d_1999={k:v*2**2 for k,v in d_1999.items()}
d_2004={k:v*2**3 for k,v in d_2004.items()}
d_2009={k:v*2**4 for k,v in d_2009.items()}
d_2014={k:v*2**5 for k,v in d_2014.items()}


In [404]:
init_dict=[d_1998,d_1999,d_2004,d_2014]
# sum the values with same keys 
counter = Counter() 
for d in init_dict:  
    counter.update(d) 
      
result = dict(counter) 
  
  
print("resultant dictionary : ", str(counter))

resultant dictionary :  Counter({'BJP': 11144, nan: 7544, 'INC': 3304, 'ADMK': 1260, 'AITC': 1136, 'CPM': 828, 'BJD': 786, 'SHS': 700, 'TDP': 692, 'SP': 580, 'TRS': 392, 'RJD': 382, 'NCP': 296, 'YSRCP': 288, 'SAD': 216, 'BSP': 214, 'JD(U)': 212, 'LJP': 192, 'DMK': 188, 'IND': 172, 'CPI': 146, 'AAAP': 128, 'PMK': 108, 'JKPDP': 104, 'JMM': 104, 'AIUDF': 96, 'BLSP': 96, 'JD(S)': 92, 'INLD': 84, 'RSP': 78, 'IUML': 64, 'AD': 64, 'MDMK': 54, 'AIMIM': 46, 'SDF': 46, 'NPF': 40, 'JKN': 38, 'KEC(M)': 38, 'RLD': 32, 'LJNSP': 32, 'SWP': 32, 'AINRC': 32, 'NPEP': 32, 'SAP': 24, 'AIFB': 24, 'MUL': 20, 'AGP': 16, 'SJP(R)': 14, 'WBTC': 14, 'JD': 12, 'FBL': 12, 'KEC': 12, 'RPI': 8, 'HLD(R)': 8, 'ABLTC': 8, 'BNP': 8, 'MNF': 8, 'IFDP': 8, 'NLP': 8, 'RPI(A)': 8, 'LS': 6, 'MSCP': 6, 'PWPI': 6, 'TMC(M)': 6, 'AC': 4, 'BBM': 4, 'CPI(ML)(L)': 4, 'MADMK': 4, 'SAD(M)': 4, 'HVC': 4, 'ASDC': 2, 'UMFA': 2, 'HVP': 2, 'AIIC(S)': 2, 'JP': 2, 'AIRJP': 2})


In [405]:
dict_len=len(result.keys())

# Creating a dataset with PARTY and Weighted Points Calculated above.

result_data=pd.DataFrame(pd.Series(result),columns=['SEATS'])
result_data.drop(result_data.index[3],inplace=True)
result_data.head()

,SEATS
TDP,692
CPI,146
BJP,11144
SHS,700
INC,3304


In [406]:
# Taking avg and calculating the Numebr of Seats a Party should win , according to historical data.

final_result=round(result_data*543/result_data.SEATS.sum()).sort_values(by='SEATS',ascending=False)
final_result

,SEATS
BJP,244.0
INC,72.0
ADMK,28.0
AITC,25.0
CPM,18.0
BJD,17.0
TDP,15.0
SHS,15.0
SP,13.0
TRS,9.0


**So as per the historical analysis I can say that BJP with some major NDA allies will get below seat**

**BJP   	244.0/
ADMK	   28.0/
BJD	        17.0/
SHS	       15.0/
JD(U)	     5.0/
SAD	       5.0/**